# Convolutional Neural Networks

## Project: Chest X-Ray Classification (Pneumonia vs Normal) 

---

### Instructions
For better understanding in this notebook some template code is already provided, and you will need to implement some additional functionalities to successfully complete this project.

> **Note for Implementation:** Sections that have **'IMPLEMENTATION'** heading, indicates that the following block needs implementations. Instructions details are provided for each implementation block and `To-DO` statments are also provided in the code cell. Before Starting to implement the functionality please read the instrcutions carefully and Pleaase do not modify or remove the template code!

---
### Project Description 

In this notebook, you will be implememnting a Convolutional Neural Network (CNN) model. You will go through different stages of data pre-processing and you will explore different methods by which we can implement a CNN model. 

The notebook is divided into following parts:

* Section 1: Import Dataset
* Section 2: Data Analysis & Pre-processing
* Section 3: Data Augumentation
* Section 4: Develop a CNN to Classify Chest X-Ray (from Scratch)
* Section 5: Develop a CNN to Classify Chest X-Ray (using Transfer Learning)
* Section 6: Comparison of CNN Models
* Section 7: Testing the best CNN Model

---
## Section 1: Import Dataset

### Chest X-Ray Dataset

For this project, we will be using [Chest X-Ray Images Dataset](https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia). This dataset consists of `5,863` X-Ray images and provides two classes of images (Normal and Pneumonia).

In the cells below, first we have imported the important libraries that will be useful throughout the notebook, we also have some constant variables like `DATASET_PATH, and IMG_CLASSES`. 

We also have a function `load_sample_imgs` which is used to load a sample training data, by which we can visualize the sample images for each class.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('/content/drive/MyDrive')

In [3]:
import re
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)
    
print(tf.__version__)

Number of replicas: 1
2.12.0


In [4]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Downloaded Dataset Path (in google drive)
DATASET_PATH = "/content/drive/MyDrive/chest_xray/"
IMG_CLASSES = ['NORMAL', 'PNEUMONIA']

def load_sample_imgs(path):
    imgs = []
    for cls in IMG_CLASSES:
        dir_path = os.path.join(path, cls)
        img_name = os.listdir(dir_path)[0]
        img = cv2.imread(os.path.join(dir_path, img_name))
        cls_index = IMG_CLASSES.index(cls)
        imgs.append([img, cls_index])
    
    return imgs

In [5]:
img_size = 224
def load_data(data_dir):
    data = [] 
    for label in IMG_CLASSES: 
        path = os.path.join(data_dir, label)
        class_num = IMG_CLASSES.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [6]:
train_data = load_data(os.path.join(DATASET_PATH, 'train'))
valid_data = load_data(os.path.join(DATASET_PATH, 'val'))

# print number of images in each dataset
print(f'There are {len(train_data)} Training Images.')
print(f'There are {len(valid_data)} Validation Images.')

<ipython-input-5-83cc3de088fd>:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data)


There are 5216 Training Images.
There are 16 Validation Images.


In [7]:
from sklearn.model_selection import train_test_split
combined_data = np.concatenate((train_data, valid_data), axis=0)
train_data, valid_data = train_test_split(combined_data, test_size=0.2, random_state=13)

# print number of images in each dataset
print(f'There are {len(train_data)} Training Images.')
print(f'There are {len(valid_data)} Validation Images.')

There are 4185 Training Images.
There are 1047 Validation Images.


---
## Section 2: Data Pre-processing

Before starting to train the Deep Learning model we need to setup the dataset for best possible results. In our case as we are working with images the first thing to note is the size of all the images in the dataset and then we can move forward with all other techniques.

In our case as we have already setup the constant value for the size of all the images in the dataset so we don't need to worry about the size of the images in the dataset. Now we have multiple other steps that we need to take, first we will going to split the features and labels from the dataset that we have prepared early.

### Implementation
In the function below provide the code to split the features and labels into their seperate lists and then return the numpy arrays for each one of them.

Now as the function is ready to be used, so we will going to use the function to split features and labels from the dataset. First we will going to call the function for training dataset and then for validation dataset.

When using TensorFlow as backend, Keras CNNs require a 4D array (which we'll also refer to as a 4D tensor) as input, with shape

$$
(\text{number_of_samples}, \text{image_height}, \text{image_width}, \text{channels}),
$$

where `number_of_samples` corresponds to the total number of images (or samples), and `image_height`, `image_width`, and `channels` correspond to the each image height, width, and channels.

By the running the cell below, we can check the tensor size of our training and validation data. First we will going to check the size of features and then we will move with labels/classes. 

### IMPLEMENTATION
For training and validation labels, we can see that we have tensors of shape `(200,)` and `(16,)`, which means we have a 1-D tensor. This is a binary dataset, which means we just have 2 classes in the dataset. For the better performance of the model we can use `One-Hot-Encoding` technique to One Hot Encode the labels, this is not a necessary step while working with Binary Classification data, but this is a cruicial step for Multi-Class Classification and that is why we also need to learn this step.

In the cell below you can implement the functionality to One-Hot Encode the labels of the dataset.

In [8]:
X_train = np.array([x[0] for x in train_data])
y_train = np.array([x[1] for x in train_data])

X_valid = np.array([x[0] for x in valid_data])
y_valid = np.array([x[1] for x in valid_data])


In [9]:
print(f'Training Features Tensor Shape: {X_train.shape}')
print(f'Validation Features Tensor Shape: {X_valid.shape}')
print('\n')
print(f'Training Labels Tensor Shape: {y_train.shape}')
print(f'Validation Labels Tensor Shape: {y_valid.shape}')

Training Features Tensor Shape: (4185, 224, 224, 3)
Validation Features Tensor Shape: (1047, 224, 224, 3)


Training Labels Tensor Shape: (4185,)
Validation Labels Tensor Shape: (1047,)


In [10]:
import numpy as np
from collections import Counter
from keras.preprocessing.image import ImageDataGenerator

# Identify the minority class
class_counts = Counter(y_train)
minority_class = min(class_counts, key=class_counts.get)

# Separate the images of the minority class
X_train_minority = X_train[y_train == minority_class]


In [11]:
# Define the data augmentation parameters
data_gen_args = dict(rotation_range=10,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=[0.9, 1.1],
                     horizontal_flip=True,
                     vertical_flip=False,
                     fill_mode='reflect',
                     data_format='channels_last')

# Create the ImageDataGenerator
image_datagen = ImageDataGenerator(**data_gen_args)

# Define the number of new samples you want to generate for the minority class
num_new_samples = 2000
augmented_images = []
augmented_labels = []

# Generate new samples using the ImageDataGenerator
for _ in range(num_new_samples):
    index = np.random.randint(0, len(X_train_minority))
    img = X_train_minority[index]
    img_augmented = image_datagen.random_transform(img)
    augmented_images.append(img_augmented)
    augmented_labels.append(minority_class)

# Convert the lists to numpy arrays
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

# Concatenate the original and augmented data
X_train_balanced = np.concatenate((X_train, augmented_images), axis=0)
y_train_balanced = np.concatenate((y_train, augmented_labels), axis=0)


In [12]:
X_train = X_train_balanced
y_train = y_train_balanced

In [13]:
# Convert labels to sequential integers
# For correctub
unique_labels = np.unique(y_train)
label_map = {label: idx for idx, label in enumerate(unique_labels)}
y_train = np.array([label_map[label] for label in y_train])
y_valid = np.array([label_map[label] for label in y_valid])


In [14]:
from keras.utils import to_categorical

# One-hot encoding for training and validation labels
y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)


Now after implementing the `One-Hot Encoding` functionality we can again check the shape of the labels.

In [15]:
print(f'Training Labels Tensor Shape: {y_train.shape}')
print(f'Validation Labels Tensor Shape: {y_valid.shape}')

Training Labels Tensor Shape: (6185, 2)
Validation Labels Tensor Shape: (1047, 2)


Now we have setup the features and labels of the dataset, let's explore the data a bit more and let us visualize the channels of the images. We know that the data is in `3 Channels` which means that the images in the dataset are having `Reg, Blue and Green` color channel which is also known as `RGB`. By running the cell below, we can visualize the image in each color channel. 

In [16]:
print(f'Training Features Tensor Shape: {X_train.shape}')
print(f'Validation Features Tensor Shape: {X_valid.shape}')
print('\n')
print(f'Training Labels Tensor Shape: {y_train.shape}')
print(f'Validation Labels Tensor Shape: {y_valid.shape}')

Training Features Tensor Shape: (6185, 224, 224, 3)
Validation Features Tensor Shape: (1047, 224, 224, 3)


Training Labels Tensor Shape: (6185, 2)
Validation Labels Tensor Shape: (1047, 2)


---
## Section 3: Data Augumentation

Data augmentation is a technique used in machine learning to artificially increase the size and diversity of a dataset by generating new data from existing data. Data augmentation provides improvement in the performance of machine learning models by exposing them to a wider range of variations in the data. Data augmentation is commonly used in computer vision applications such as image classification, object detection, and segmentation. 

For data augumentation we can make use of multiple techniques like Flipping, Rotation, Scaling, Cropping, Translation, Adding Noise to the image, etc. 

### IMPLEMENTATION
For Data Augumentation we will be using `ImageDataGenerator` function provided by `keras` library. In the cell below you have boilerplate code for the initialization of ImageDataGenerator object, while initializing the object, we need to pass in arguments which Augumentation techniques we need to use. 

**In the code below, we already have two arguments (Augumentation Techniques), you need to provide at least 2 more arguments to Augument the data, you can provide more as you want.** 

In [17]:
from keras.preprocessing.image import ImageDataGenerator

# TODO: Provide at least 2 more data augumentation parameters,
# we need to apply at least 4 different augumentation techniques.
datagen = ImageDataGenerator(rescale=1./255, rotation_range=10, width_shift_range=0.1, height_shift_range=0.1 ,brightness_range=(0.8,1), zoom_range=0.2)

Now after initializing the data augumentation object, we need to use this object to visualize the dataset. 

In the cell below, we will use one single sample image and generate the augumented images. The generator object will augument the sample image and generate different samples of the images by the help of the techniques which we have provided. 

---
## Section 4: Develop a CNN to Classify Chest X-Ray (from Scratch)

We have completed the Data pre-processing part, now it is the time to start developing the Convolutional Neural Network model. For this project we will be going to work with keras and Tensorflow library to develop and train the CNN model. CNN models have multiple layers that we can use, some of the standard layers that we need to use while working with CNN models are `Convolutinal Layer, Pooling Layer, Dropout Layer, Flatten Layer and Dense Layer`.

### IMPLEMENTATION
Now in the cell below we will going to define our model architecture. You have been given a boiler plate code with 1 Convolutional Layer and Output Layer of the model. 

**By using the boilerplate code develop a Convolutional Model architecutre. You need to use Conv2D, Pooling, Dropout and Dense Layers, etc. Please don't change anything in the boilerplate code.**

In [ ]:
img_size=224
# deeper structure with fewer parameter
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

# Building CNN model
# Initializing the Model
cnn_model = Sequential()

# Conv Layer
cnn_model.add(Conv2D(filters=16, kernel_size=3, padding="same", activation="relu", input_shape=(img_size, img_size, 3)))
# Pooling layer
cnn_model.add(MaxPooling2D(pool_size=2))
# Dropout layer
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"))
cnn_model.add(MaxPooling2D(pool_size=2))
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
cnn_model.add(MaxPooling2D(pool_size=2))
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(filters=128, kernel_size=3, padding="same", activation="relu"))
cnn_model.add(MaxPooling2D(pool_size=2))
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(filters=256, kernel_size=3, padding="same", activation="relu"))
cnn_model.add(MaxPooling2D(pool_size=2))
cnn_model.add(Dropout(0.25))


# Flatten layer
cnn_model.add(Flatten())

# Dense layers
cnn_model.add(Dense(128, activation="relu"))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(64, activation="relu"))
cnn_model.add(Dropout(0.5))

# Output Layer of the Model
cnn_model.add(Dense(2, activation="softmax"))

# Model Summary
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 16)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 112, 112, 16)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 56, 56, 32)        0

In [ ]:
img_size=224
# deeper structure with fewer parameter
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

# Building CNN model
# Initializing the Model
cnn_model = Sequential()

# Conv Layer
cnn_model.add(Conv2D(filters=16, kernel_size=3, padding="same", activation="relu", input_shape=(img_size, img_size, 3)))
# Pooling layer
cnn_model.add(MaxPooling2D(pool_size=2))
# Dropout layer
cnn_model.add(Dropout(0.1))

cnn_model.add(Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"))
cnn_model.add(MaxPooling2D(pool_size=2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
cnn_model.add(MaxPooling2D(pool_size=2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv2D(filters=128, kernel_size=3, padding="same", activation="relu"))
cnn_model.add(MaxPooling2D(pool_size=2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv2D(filters=256, kernel_size=3, padding="same", activation="relu"))
cnn_model.add(MaxPooling2D(pool_size=2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv2D(filters=512, kernel_size=3, padding="same", activation="relu"))
cnn_model.add(MaxPooling2D(pool_size=2))
cnn_model.add(Dropout(0.2))


# Flatten layer
cnn_model.add(Flatten())

# Dense layers
cnn_model.add(Dense(128, activation="relu"))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(64, activation="relu"))
cnn_model.add(Dropout(0.5))

# Output Layer of the Model
cnn_model.add(Dense(2, activation="softmax"))

# Model Summary
cnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 224, 224, 16)      448       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 112, 112, 16)     0         
 2D)                                                             
                                                                 
 dropout_7 (Dropout)         (None, 112, 112, 16)      0         
                                                                 
 conv2d_6 (Conv2D)           (None, 112, 112, 32)      4640      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 56, 56, 32)       

The output of the above cell shows the summary of the CNN model. With the help of the model summary, we can see the trainable parameters, the complexity of the model, we can also check the output data shape for each layer. This is a good way to check the complexity of the model. 

Now we need to compile the model. While compiling the model we need to provide `loss function`, `optimizer`, and `metrics` which will be used during the training process of the Deep Learning model. There are many different choices of loss functions and optimizers which are avilable and we need to choose the best for our use case. 

### IMPLEMENTATION
In the cell below, **you need to provide the LOSS function and Optimization Function for the CNN Model**.

- [Available Loss Functions](https://keras.io/api/losses/probabilistic_losses/)
- [Available Optimization Functions](https://keras.io/api/optimizers/)

In [ ]:
# when optimizer is "adam", best model Training Accuracy: 77.873 % loss 24.537975311279297
# improve it by SGD

In [ ]:
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau
sgd_optimizer = SGD(learning_rate=0.01, momentum=0.9, decay=5e-4)

In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)


In [ ]:
cnn_model.compile(loss="binary_crossentropy", optimizer=sgd_optimizer, metrics=["accuracy"])

NameError: ignored

### IMPLEMENTATION

Now in the cell below we will be going to train our Convolutional Neural Network Model. For the training of the CNN model, **you need to provide a `Batch size` and `Number of Epochs`**. And after that you can run this cell to start the training process of the model.

In [ ]:
#!pip install --upgrade tensorflow keras

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("best_model_epoch_{epoch:02d}.h5", monitor="val_accuracy", save_best_only=True, mode="max", verbose=1, save_freq='epoch')

In [ ]:
Epochs = 20 
BATCH_SIZE = 20

cnn_model_history = cnn_model.fit(X_train, y_train, batch_size=BATCH_SIZE,
                                  validation_data=(X_valid, y_valid), epochs=Epochs,
                                  verbose=1, shuffle=True, callbacks=[checkpoint])

Epoch 1/20
310/310 [==============================] - ETA: 0s - loss: nan - accuracy: 0.4944
Epoch 1: val_accuracy improved from -inf to 0.27698, saving model to best_model_epoch_01.h5


KeyboardInterrupt: ignored

#### Evaluating the Trained Model
After training the Model, it is a good paractice to evaluate the model on Training Dataset to get the final Training Accuracy and Loss of the Model. By running the cell below you will be able to find out the training accuracy and loss of the CNN model. 

In [ ]:
# use batchnormalize achieved 74
# use batch+sgd 25
# best to now: 2 million parameter 87.599 %
#Training Accuracy: 87.599 %
#Training Loss: 9.101021766662598

In [ ]:
cnn_model.load_weights("best_model_epoch_01.h5")

training_score = cnn_model.evaluate(X_train, y_train)

print("\nTraining Accuracy:", round((training_score[1] * 100), 3),"%")
print("Training Loss:", training_score[0])

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
y_pred_probs = cnn_model.predict(X_train)
y_pred_labels = np.argmax(y_pred_probs, axis=1)
y_true_labels = np.argmax(y_train, axis=1)
cm = confusion_matrix(y_true_labels, y_pred_labels)
print("Confusion Matrix:")
print(cm)


In [ ]:
img_size = 224
def load_data(data_dir):
    data = [] 
    path = data_dir
    for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append([resized_arr, img])
            except Exception as e:
                print(e)
    return np.array(data)

test_data = load_data(os.path.join(DATASET_PATH, 'test'))

# print number of images in each dataset
print(f'There are {len(test_data)} Training Images.')

X_test = np.array([x[0] for x in test_data])
y_test = np.array([x[1] for x in test_data])

print(f'Training Features Tensor Shape: {X_test.shape}')
print(f'Training Features Tensor Shape: {y_test.shape}')

y_test_prob = cnn_model.predict(X_test)
y_test_pred = np.argmax(y_test_prob, axis=1)

results_df = pd.DataFrame({"ID":y_test,"class": y_test_pred})
df = results_df
df['class'] = df['class'].map({0: 'NORMAL', 1: 'PNEUMONIA'})
df['ID_number'] = df['ID'].str.extract('(\d+)').astype(int)

df_sorted = df.sort_values(by='ID_number')
df_sorted.drop('ID_number', axis=1, inplace=True)

df_sorted.to_csv("predicted_labels.csv", index=False)

from google.colab import files

files.download("predicted_labels.csv")

---
## Section 5: Develop a CNN to Classify Chest X-Ray (using Transfer Learning)

Now in this section we will be using pre-trained CNN model and train it using Transfer Learning Techniques. Transfer learning is a powerfull technique by which we can make use of complex and large pre-trained models and improve those models on our datasets. We have many different pre-trained CNN models available, you can access the list of all pre-trainned models in Keras Document, [link to pre-trained models](https://keras.io/api/applications/). 

There are different types of Transfer Learning techniques available, for this project we will be using `Fine Tuning` approach. Fine-tuning in transfer learning is a technique which is used to adapt a pre-trained model to a new dataset. This involves taking a pre-trained model that has been trained on a large dataset, and then improving it for a different dataset. The idea is that the model has already learned some general features and patterns from the original dataset, and these can be useful for the new dataset.

### IMPLEMENTATION

The First step to implement the Fine Tuning approach is to initialize a pre-trained model. **From the list of all available pre-trained models, choose a model according to your choice and initialize it in the cell given below**.

In [36]:
# TODO: Initialize a pre-trained model
import tensorflow
from tensorflow import keras
from tensorflow.keras.applications import EfficientNetV2S

eff_pretrained_model = EfficientNetV2S(include_top = False, weights ='imagenet',  input_shape = (224,224,3))

# We don't need to train the pre-trained model we just need to fine-tune it

for i, layer in enumerate(eff_pretrained_model.layers):
    if i < len(eff_pretrained_model.layers) - 2:
        layer.trainable = False
    else:
        layer.trainable = True
# Note that setting layer.trainable = True will train model from scratch. 
# In this case, only the architechure will be the same as your pretrained models

Now in the second step of Fine Tuning, we need to add some `Dense` layers to the model. As you can see in the cell above, we have explicitly turned off the training of all the layers inside the pre-trainned model because we don't want the pre-trained model to be trained we just want to add some layers at the end of the model and then we will going to train those layers.

### IMPLEMENTATION
In the cell below, you have given a boilerplate code, the loaded pre-trained model has been attached to the CNN architecture and also the output layer of the model is initialized. **You need to add some more Dense, Dropout Layers in the CNN model architecture**.

In [37]:
#EFFICIENTNET
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
eff_transfer_model = Sequential()     
eff_transfer_model.add(eff_pretrained_model)
eff_transfer_model.add(Flatten())
# Hint: Add a pooling layer to reduce the input size to dense layer.

#TODO: Add some Dense, Dropout Layers
eff_transfer_model.add(Dense(128, activation='relu'))
eff_transfer_model.add(Dropout(0.3))
eff_transfer_model.add(Dense(16, activation='relu'))
eff_transfer_model.add(Dropout(0.3))
# Hint: Keep your size of Dense layer small (i.e. number of units in the dense layers. You may find colab crashing if you have too many params.)
# Output Layer of the Model
eff_transfer_model.add(Dense(2, activation='softmax'))
eff_transfer_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-s (Functiona  (None, 7, 7, 1280)       20331360  
 l)                                                              
                                                                 
 flatten_2 (Flatten)         (None, 62720)             0         
                                                                 
 dense_7 (Dense)             (None, 128)               8028288   
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 16)                2064      
                                                                 
 dropout_6 (Dropout)         (None, 16)                0         
                                                      

In [18]:
#EFFICIENTNET
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras import layers
eff_transfer_model = Sequential()     
eff_transfer_model.add(eff_pretrained_model)
eff_transfer_model.add(Flatten())
# Hint: Add a pooling layer to reduce the input size to dense layer.
#TODO: Add some Dense, Dropout Layers
eff_transfer_model.add(Dense(512, activation='relu'))
eff_transfer_model.add(Dropout(0.3))
eff_transfer_model.add(Dense(128, activation='relu'))
eff_transfer_model.add(Dropout(0.3))
eff_transfer_model.add(Dense(32, activation='relu'))
eff_transfer_model.add(Dropout(0.3))
# Hint: Keep your size of Dense layer small (i.e. number of units in the dense layers. You may find colab crashing if you have too many params.)
# Output Layer of the Model
eff_transfer_model.add(Dense(2, activation='softmax'))
eff_transfer_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-s (Functiona  (None, 7, 7, 1280)       20331360  
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 62720)             0         
                                                                 
 dense (Dense)               (None, 512)               32113152  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               65664     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                        

In [38]:
#TODO: Provide Loss and Optimizer functions
#!!!!!!!!!!!!!
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
eff_transfer_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


### IMPLEMENTATION

Now in the cell below we will be going to train the Convolutional Neural Network Model. As we have trained the CNN model previously, it will going to be the same steps. **Provide the code for training the CNN Model, and train the model on same number of Epochs as the previous CNN model**.

In [39]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("best_model_epoch_{epoch:02d}.h5", monitor="val_accuracy", save_best_only=True, mode="max", verbose=1, save_freq='epoch')

In [33]:
from keras.preprocessing.image import ImageDataGenerator

# TODO: Provide at least 2 more data augumentation parameters,
# we need to apply at least 4 different augumentation techniques.
datagen = ImageDataGenerator(rescale=1./255, rotation_range=10, zoom_range=0.05)

In [ ]:
#TODO: You need to provide the Batch Size, Number of Epochs will be the same as before
BATCH_SIZE = 32

eff_transfer_model_history = eff_transfer_model.fit(datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),\
                                            validation_data=(X_valid, y_valid), epochs=10,\
                                            verbose=1, shuffle=True, callbacks=[checkpoint])

In [40]:
#TODO: You need to provide the Batch Size, Number of Epochs will be the same as before
BATCH_SIZE = 20

eff_transfer_model_history = eff_transfer_model.fit(X_train, y_train, batch_size=BATCH_SIZE,\
                                            validation_data=(X_valid, y_valid), epochs=10,\
                                            verbose=1, shuffle=True, callbacks=[checkpoint])

Epoch 1/10
309/310 [============================>.] - ETA: 0s - loss: 0.3019 - accuracy: 0.8916
Epoch 1: val_accuracy improved from -inf to 0.88539, saving model to best_model_epoch_01.h5
310/310 [==============================] - 71s 177ms/step - loss: 0.3019 - accuracy: 0.8915 - val_loss: 0.1743 - val_accuracy: 0.8854
Epoch 2/10
309/310 [============================>.] - ETA: 0s - loss: 0.1760 - accuracy: 0.9371
Epoch 2: val_accuracy improved from 0.88539 to 0.91213, saving model to best_model_epoch_02.h5
310/310 [==============================] - 52s 169ms/step - loss: 0.1758 - accuracy: 0.9371 - val_loss: 0.1272 - val_accuracy: 0.9121
Epoch 3/10
309/310 [============================>.] - ETA: 0s - loss: 0.1232 - accuracy: 0.9566
Epoch 3: val_accuracy improved from 0.91213 to 0.93505, saving model to best_model_epoch_03.h5
310/310 [==============================] - 50s 163ms/step - loss: 0.1231 - accuracy: 0.9567 - val_loss: 0.1030 - val_accuracy: 0.9351
Epoch 4/10
309/310 [========

In [41]:
eff_transfer_model.load_weights("best_model_epoch_09.h5")
training_score = eff_transfer_model.evaluate(X_train, y_train)
print("\nTraining Accuracy:", round((training_score[1] * 100), 3), "%")
print("Training Loss:", training_score[0])

194/194 [==============================] - 23s 118ms/step - loss: 0.0060 - accuracy: 0.9984

Training Accuracy: 99.838 %
Training Loss: 0.005994182080030441


In [42]:
training_score = eff_transfer_model.evaluate(X_valid, y_valid)
print("\nTraining Accuracy:", round((training_score[1] * 100), 3), "%")
print("Training Loss:", training_score[0])

33/33 [==============================] - 4s 119ms/step - loss: 0.0737 - accuracy: 0.9809

Training Accuracy: 98.09 %
Training Loss: 0.0736745148897171


In [43]:
from sklearn.metrics import confusion_matrix
import numpy as np
y_pred_probs = eff_transfer_model.predict(X_train)
y_pred_labels = np.argmax(y_pred_probs, axis=1)
y_true_labels = np.argmax(y_train, axis=1)
cm = confusion_matrix(y_true_labels, y_pred_labels)
print("Confusion Matrix:")
print(cm)

194/194 [==============================] - 25s 112ms/step
Confusion Matrix:
[[3065    1]
 [   9 3110]]


In [44]:
img_size = 224
def load_data(data_dir):
    data = [] 
    path = data_dir
    for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append([resized_arr, img])
            except Exception as e:
                print(e)
    return np.array(data)

test_data = load_data(os.path.join(DATASET_PATH, 'test'))

# print number of images in each dataset
print(f'There are {len(test_data)} Training Images.')

X_test = np.array([x[0] for x in test_data])
y_test = np.array([x[1] for x in test_data])

print(f'Training Features Tensor Shape: {X_test.shape}')
print(f'Training Features Tensor Shape: {y_test.shape}')

y_test_prob = eff_transfer_model.predict(X_test)
y_test_pred = np.argmax(y_test_prob, axis=1)

results_df = pd.DataFrame({"ID":y_test,"class": y_test_pred})
df = results_df
df['class'] = df['class'].map({0: 'NORMAL', 1: 'PNEUMONIA'})

df['ID_number'] = df['ID'].str.extract('(\d+)').astype(int)

df_sorted = df.sort_values(by='ID_number')
df_sorted.drop('ID_number', axis=1, inplace=True)

df_sorted.to_csv("predicted_labels.csv", index=False)

from google.colab import files

files.download("predicted_labels.csv")

<ipython-input-44-dd2f3f6ef981>:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data)


There are 624 Training Images.
Training Features Tensor Shape: (624, 224, 224, 3)
Training Features Tensor Shape: (624,)
20/20 [==============================] - 2s 111ms/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# Create a pandas DataFrame with the predicted labels
results_df = pd.DataFrame({"ID":y_test,"Label": y_test_pred})

# Save the DataFrame to a CSV file
results_df.to_csv("predicted_labels.csv", index=False)

from google.colab import files

files.download("predicted_labels.csv")



In [ ]:
df = results_df
df

In [ ]:
df['Label'] = df['Label'].map({0: 'NORMAL', 1: 'PNEUMONIA'})

In [ ]:
df

In [ ]:
# Extract the integer part from the 'ID' column
df['ID_number'] = df['ID'].str.extract('(\d+)').astype(int)

# Sort the dataframe based on the extracted number
df_sorted = df.sort_values(by='ID_number')

# Drop the temporary 'ID_number' column
df_sorted.drop('ID_number', axis=1, inplace=True)

In [ ]:
df_sorted

In [ ]:
# Save the DataFrame to a CSV file
df_sorted.to_csv("predicted_labels.csv", index=False)

from google.colab import files

files.download("predicted_labels.csv")

#### Evaluating the Trained Model
After training the Model, it is a good paractice to evaluate the model on Training Dataset to get the final Training Accuracy and Loss of the Model. By running the cell below you will be able to find out the training accuracy and loss of the CNN model which is trained by transfer learning. 

In [ ]:
res_transfer_model.load_weights("best_model.h5")
transfer_model_score = res_transfer_model.evaluate(X_train, y_train)

print("\nResNetTraining Accuracy:", round((transfer_model_score[1] * 100), 3),"%")
print("ResNet Training Loss:", transfer_model_score[0])


inc_transfer_model.load_weights("best_model.h5")
transfer_model_score = inc_transfer_model.evaluate(X_train, y_train)

print("\nInception Training Accuracy:", round((transfer_model_score[1] * 100), 3),"%")
print("Inception Training Loss:", transfer_model_score[0])


den_transfer_model.load_weights("best_model.h5")
transfer_model_score = den_transfer_model.evaluate(X_train, y_train)

print("\nDenseNet Training Accuracy:", round((transfer_model_score[1] * 100), 3),"%")
print("DenseNet Training Loss:", transfer_model_score[0])



transfer_model_score = eff_transfer_model.evaluate(X_train, y_train)

print("\nEfficientNet Training Accuracy:", round((transfer_model_score[1] * 100), 3),"%")
print("EfficientNet Training Loss:", transfer_model_score[0])

---
## Section 6: Comparison of CNN Models

### Visualizing Model Loss And Accuracy

We have successfully trained both CNN models and now it's time to compare the accuracy and loss of both models to know which one is the best one. While training the models, we can see that we have used **cnn_model_history variable** and **transfer_model_history variable**, these variable are used to store the accuracy and loss of the model for all Epochs. These variable holds the data in the form of python dictionary.

### IMPLEMENTATION
**By using the `cnn_model_history` and `transfer_model_history` variables and matplotlib library plot the Accuracy and Loss graphs of the models. You need to plot Accuracy and Loss graphs having Training and Validation data for both models.**

In [ ]:
#TODO: Plot the Accuracy Graph

In [ ]:
#TODO: Plot the Loss Graph

---
## Section 7: Testing the best CNN Model

Now in this last section of the Notebook, we will test the best CNN model.

#### IMPLEMENTATION
**Load the testing set in the cell below.**

In [ ]:
# TODO: Load the Testing Dataset
test_data = None

# print number of images in testing dataset
print(f'There are {len(test_data)} Testing Images.')

### IMPLEMENTATION
**Split the Features and Labels of the Testing Set.**

In [ ]:
# TODO: Split the Features and Labels of Testing Data
x_test, y_test = None

print(f'Testing Features Tensor Shape: {x_test.shape}')

We have loaded and setup the testing dataset, now it's time to use the best model and make the prediction for testing images.

In [ ]:
predictions = transfer_model.predict(x_test)
prediction_labels = np.argmax(predictions, axis=1)

### Confusion Matrix
A confusion matrix is a table used to evaluate the performance of a classification model. It shows the number of correct and incorrect predictions made by the model compared to the actual outcomes. The matrix is usually presented as a square table, with the rows representing the actual class labels and the columns representing the predicted class labels. The confusion matrix provides several useful metrics for evaluating the performance of a classification model, such as accuracy, precision, recall, and F1 score. 

### IMPLEMENTATION
We have original labels and predicted labels, now you need to use **scikit-learn** and **matplotlib** lilbrary to plot the confusion matrix.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, classification_report

#TODO: Compute and Plot the Confusion matrix for testing data

### Classification Report

A classification report is a summary of the key performance metrics for a classification model. The classification report includes several important metrics, including precision, recall, F1 score, and support. These metrics provide information about the model's ability to correctly identify positive and negative instances, as well as the balance between precision and recall.

Please run the cell below to generate the classification report for the Best Performing CNN Model.

In [ ]:
print(classification_report(y_test, prediction_labels))

---
## Conclusion

In the Notebook, we have learned how we can develop Convolutional Neural Network Models. We have learned how we can develop our own CNN architectures and how we can make use of Transfer Learning Techniques. We have gone through the data preperation part, model development part and finally we have evaluated and tested the CNN Models. 